# **ETL**

In [2]:
# Main libraries

import requests 
import os 
import io 
import pandas as pd  
import bs4 
import numpy as np
from PIL import Image
import sqlalchemy
from sqlalchemy import create_engine



In [19]:
pd.set_option('display.max_rows', 5)

### **Weather icons**

**Method**: web scraping

In [20]:
# Connection to get the Icon list table: https://openweathermap.org/weather-conditions

URL =  'https://openweathermap.org/weather-conditions'
req = requests.get(URL)
if req.status_code != 200:
    print(f"Error: status code {req.status_code}")


In [21]:
# Define data frame

soup = bs4.BeautifulSoup(req.text, 'html.parser')
table = soup.find_all('table')[0]

headers = table.find_all('th')
for i in range(len(headers)):
    headers[i] = headers[i].text.strip() 

rows = table.find_all('tr')
for i in range(len(rows)):
    rows[i] = rows[i].find_all('td')
    for j in range(len(rows[i])):
        rows[i][j]=rows[i][j].text.strip()
        

weather_icons=pd.DataFrame(rows[1:], columns=['day_icon', 'night_icon', 'description'])
weather_icons



,day_icon,night_icon,description
0,01d.png,01n.png,clear sky
1,02d.png,02n.png,few clouds
...,...,...,...
7,13d.png,13n.png,snow
8,50d.png,50n.png,mist


In [22]:
# Saving the table
file_path =  os.getcwd() + '/../cleaned_data/weather_icons.csv'
weather_icons.to_csv(file_path, index=False)

In [23]:
# save images 
img_dir =  os.getcwd() + '\\..\\img\\'

if not os.path.exists(img_dir):
    os.mkdir(img_dir)
    print("Folder created successfully!")
else:
    print("Folder already exists.")

images = table.find_all('img')
names=[]
for image in images:
    image_url=image.get('src')
       
    response = requests.get(image_url)
    img_data = response.content 

    name =  image_url.rsplit('/',1)[-1]
    name = name[:3]

    img = Image.open(io.BytesIO(img_data))
    img.save(img_dir+ name+'.png')
    



Folder already exists.


In [ ]:
# Database Connection: We use sql server with a trusted connection and weather as database name
engine = create_engine('mssql+pyodbc://SERVER_NAME/weather?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')



In [ ]:
# Database load
weather_icons.to_sql('weather_icons', engine, if_exists='append', index=False)
